<a href="https://www.kaggle.com/metin2/sentiment-analysis?scriptVersionId=82515310" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Model Creation 
---

In [ ]:
!pip install opendatasets
!pip install nltk
!pip install TurkishStemmer
!pip install sklearn
!pip install keras
!pip install tensorflow
!pip install googletrans==4.0.0-rc1

In [ ]:
import opendatasets as od
od.download("https://www.kaggle.com/metin2/sentimentanalysis")

In [2]:
import ssl
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context
import codecs
import os
import re 
import pandas as pd
import seaborn as sn
import numpy as np
import tensorflow as tf
import codecs
import string 
import nltk
from nltk.tokenize import TweetTokenizer
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')

from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
#df = pd.read_csv('sentimentAnalysis.csv')
df = pd.read_csv('../input/sentimentanalysis/sentimentAnalysis.csv')
df[["TranslatedText","SentimentAnalysis"]].head(10)

#when SentimentAnalysis=0 then 'Ignore'
#when SentimentAnalysis=1 then 'Negative'
#when SentimentAnalysis=2 then 'Neutral'
#when SentimentAnalysis=3 then 'Positive' end as SentimentAnalysis

,TranslatedText,SentimentAnalysis
0,Use omo get immaculate,2
1,2 times wear and bury the closet 😂,2
2,Several Different Several Different Boylece Ol...,2
3,"Low degree, use little ..",2
4,Legend I don't believe something,2
5,I think the most useful and useful rinso.🙂👍,3
6,Don't need to have a lot of reversed and have ...,2
7,Health thanks to your uniform,2
8,HEALTHY INFORMATION HEALTH HEALTH,0
9,A correct degree and a good detergent,2


In [4]:
def textCleaning(text):
    # lower
    text = text.lower()

    # remove hyperlinks
    text = re.sub(r"<.*?a>", "", text)

    # remove html
    text = re.sub(r"<[^>]+>", "", text)

    # remove urls
    text = re.sub(r"(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,})", "", text)
    
    # encoding the text - cleaning the emoji
    text_encode = text.encode(encoding="utf-8", errors="ignore")
    # text_encode = text.encode(encoding="latin5", errors="ignore")

    # decoding the text
    text_decode = text_encode.decode("ascii", errors="ignore")
    # text_decode = text_encode.decode("latin5", errors="replace_with_space")
    text = text_decode
    
    # Remove ticks and the next character
    text = re.sub(r"\'\w+", "", text)

    # Remove ticks and the next character
    text = re.sub(r"\'\w+", "", text)

    # cleaning the text to remove extra whitespace 
    text = re.sub(r'\s{2,}', " ", text)

    # # removing mentions 
    text = re.sub(r"@\S+", "", text)

    # # Remove Hashtags
    text = re.sub(r"#\S+", "", text)
    
    # remove market tickers
    text = re.sub(r"\$", "", text)

    # remove punctuation
    stringPunct= '''!"$%&'()*+,-./:;<=>?[\]^_`{|}~'''
    text = text.translate(str.maketrans('', '',stringPunct))

    # Remove numbers
    text = re.sub(r'\w*\d+\w*', '', text)

    # Tokenizer
    tt = TweetTokenizer()
    words = tt.tokenize(text)

    #hastags = []
    #[hastags.append(token) for token in words if token.startswith("#") ]

    #mentions = []
    #[mentions.append(token) for token in words if token.startswith("@") ]

    words = [word for word in words if word not in stopwords.words('english')]
    #the stemmer requires a language parameter
    snow_stemmer = SnowballStemmer(language='english')
    
    #stem's of each word
    stem_words = []
    for w in words:
        x = snow_stemmer.stem(w)
        stem_words.append(x)

    return (stem_words)

#print(stopwords.words('english'))
#textCleaning("I always tell my female clients to chat/text for a day or so then move it to")

In [5]:
X=df["TranslatedText"].tolist()
y=df["SentimentAnalysis"].tolist()
#train, test = train_test_split(df, test_size=0.2, random_state=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

Text_X_train, Text_X_test, Text_y_train, Text_y_test = X_train, X_test, y_train, y_test 

y_train = np.array(y_train)

In [6]:
print(X_train[0])
print(y[0])

print(len(X_train))
print(len(y_train))

Bi times a week stir and take care and take care of the eeeen beauty as well.I never do the paint ❤ and I immediately wash my hair off the sea.
2
183635
183635


In [ ]:
max_features = 7000

tokenized = X_train
#df1["AnswerText"].tolist()

vectorizer = TfidfVectorizer(analyzer='word',
                        tokenizer=textCleaning,
                        vocabulary=None,
                        # preprocessor=dummy_fun,
                        lowercase = False,
                        use_idf = True,
                        token_pattern=None,
                        ngram_range=(1,3),
                        max_df=0.9, min_df=2,
                        max_features=max_features
                    )
tfs = vectorizer.fit_transform(tokenized).todense()

In [16]:
vocabulary = vectorizer.vocabulary_
# print(vocabulary)
# print(vectorizer.get_feature_names_out())
print(len(vocabulary))

7000


In [ ]:
print(tfs.shape)
print(y_train.shape)

In [18]:
X_test = vectorizer.transform(X_test).todense()
y_test = np.array(y_test)

In [ ]:
#initial model
model = keras.Sequential([
    keras.layers.Dense(128, input_shape=(7000,),activation='relu'),
    keras.layers.Dense(64),
    keras.layers.Dense(16),
    keras.layers.Dense(4,activation='sigmoid')
])
model.summary()

In [20]:
#compiling of the model
model.compile(
    optimizer='adam',
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy']
)

model.fit(tfs, y_train, epochs=5)

2021-12-16 12:01:23.308551: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/5
5739/5739 [==============================] - 34s 6ms/step - loss: 0.6389 - accuracy: 0.7645
Epoch 2/5
5739/5739 [==============================] - 31s 5ms/step - loss: 0.5629 - accuracy: 0.7864
Epoch 3/5
5739/5739 [==============================] - 33s 6ms/step - loss: 0.5100 - accuracy: 0.8024
Epoch 4/5
5739/5739 [==============================] - 33s 6ms/step - loss: 0.4420 - accuracy: 0.8261
Epoch 5/5
5739/5739 [==============================] - 30s 5ms/step - loss: 0.3728 - accuracy: 0.8537


In [ ]:
model.save("sentimentAnalysis.h5")
import pickle
filename="vectorizer"
with open(filename, 'wb') as fp:
    pickle.dump(vectorizer, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [21]:
model.evaluate(X_test, y_test)

2460/2460 [==============================] - 6s 2ms/step - loss: 0.7800 - accuracy: 0.7681


[0.7800027132034302, 0.7680715322494507]

In [ ]:
y_predicted = model.predict(X_test)
y_predicted[0]

In [ ]:
print("Text: ", Text_X_test[0])
print("label: ",np.argmax(y_predicted[0]))

In [ ]:
#Draw a graph
y_predicted = model.predict(X_test)
y_predicted_labels = [np.argmax(i) for i in y_predicted]
cm = tf.math.confusion_matrix(labels=y_test, predictions= y_predicted_labels )
plt.figure(figsize = (10,7))
sn.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Truth')

In [ ]:
text = ["I'm sleeping regularly 8 hours for health and I'm considering very balanced nutrition.I also use regular Oalrak food supplies","it is ok, love it"]
x = vectorizer.transform(text).todense()
np.argmax(model.predict(x)[0])

# Test Part
---

In [ ]:
from tensorflow import keras
import pickle

m = keras.models.load_model("sentimentAnalysis.h5")

with open("vectorizer", 'rb') as pfile:
    v = pickle.load(pfile)

In [ ]:
text = ["I'm sleeping regularly 8 hours for health and I'm considering very balanced nutrition.I also use regular Oalrak food supplies","it is ok, love it"]
x = v.transform(text).todense()
np.argmax(m.predict(x)[0])